In [2]:
#Se conecta Google Colaboratory con Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Instala pyspark en Google Colaboratory
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=15770e560dec868541c78da78ece7108eb34be78e8e077b0ca59a4a0578b0e40
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [4]:
#Se importan las librerias
import pandas as pd
import numpy as np
import ast
import json
import pickle
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, explode, sum,expr
from pyspark.sql.types import StructType, StructField, StringType, IntegerType


In [5]:
spark = SparkSession.builder.appName("LecturaDeArchivos").getOrCreate()

In [6]:
#Se cargan los archivos de la carpeta metadata-sitios y se compilan en un solo DataFrame

metadata_sitios=[]

for i in range(1,12):


    archivo = spark.read.json(f"/content/drive/My Drive/Google Maps/metadata-sitios/{i}.json")

    archivo = archivo.withColumn("MISC", col("MISC").cast("string"))

    metadata_sitios.append(archivo)

df_final = metadata_sitios[0]

for dataframe in metadata_sitios[1:]:

  df_final = df_final.unionByName(dataframe)


metadata_sitios=df_final

In [7]:
metadata_sitios.show()

+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+------------------+-------------------+--------------------+--------------+-----+--------------------+--------------------+--------------------+
|                MISC|             address|avg_rating|            category|         description|             gmap_id|               hours|          latitude|          longitude|                name|num_of_reviews|price|    relative_results|               state|                 url|
+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+------------------+-------------------+--------------------+--------------+-----+--------------------+--------------------+--------------------+
|{[Wheelchair acce...|Porter Pharmacy, ...|       4.9|          [Pharmacy]|                NULL|0x88f16e41928ff68...|[[Friday, 8AM–6PM...|           32

Comenzamos a analizar este DataFrame sobre los diferentes negocios en los Estados Unidos, entre los datos podemos encontrar diferentes categorías como farmacias, restaurantes, cafeterias, gimnasios,... etc. Para nuestro proyecto es importante enfocarnos en el sector gastronómico, sin embargo en este Análisis de datos preliminar abarcaré el estudio de la calidad de los datos en general. En primera instancia, no tenemos una columna que indique explícitamente el Estado en el que se encuentra el negocio, sin embargo a futuro podemos construir una función que nos proporcione ese dato a partir de las coordenadas.

Comenzamos contando el número de registros de nuestro dataset para tener una dimensión del conjunto de datos que manejamos: 3025011. Ahora es importante conocer qué cantidad de datos faltan en cada columna para saber si es proporcionalmente admisible.



In [8]:
#Muestra el número de filas del DataFrame
metadata_sitios.count()

3025011

In [9]:
# Cuenta el número de nulos en cada columna

def conteo_nulos(dataframe):
  conteo_nulos_por_columna = dataframe.agg(*[sum(col(c).isNull().cast("int")).alias(c) for c in dataframe.columns])

  # Muestra el resultado
  conteo_nulos_por_columna.show()

In [10]:
conteo_nulos(metadata_sitios)

+------+-------+----------+--------+-----------+-------+------+--------+---------+----+--------------+-------+----------------+------+---+
|  MISC|address|avg_rating|category|description|gmap_id| hours|latitude|longitude|name|num_of_reviews|  price|relative_results| state|url|
+------+-------+----------+--------+-----------+-------+------+--------+---------+----+--------------+-------+----------------+------+---+
|690834|  80511|         0|   17419|    2770722|      0|787405|       0|        0|  37|             0|2749808|          295058|746455|  0|
+------+-------+----------+--------+-----------+-------+------+--------+---------+----+--------------+-------+----------------+------+---+



Como se puede observar los campos 'description' y 'price' tienen demasiados datos nulos en proporción al total de registros como para ser siquiera considerados. También otras columnas como 'state' y 'MISC' no son de relevancia para nuestro estudio, pero esto se analizará más adelante cuando tengamos que hacer el ETL y normalizar los Datasets.

Ahora nos interesa conocer las categorías de negocio en específico que contienen nuestros datos.

In [14]:
# Explota la columna 'category' y obtiene elementos únicos
categorias_sin_duplicados = metadata_sitios.select(explode("category").alias("categoria")).distinct()

# Colecta los resultados en una lista
lista_categorias_sin_duplicados = [row.categoria for row in categorias_sin_duplicados.collect()]

# Muestra la lista resultante
print("Categorías sin duplicados:", lista_categorias_sin_duplicados)

# Muestra cuantas categorías hay en total

print("El número de categorías es: ", len(lista_categorias_sin_duplicados))

#Ahora voy a contar cuantas de esas categorías son de restaurantes

contador=0

for elemento in lista_categorias_sin_duplicados:

  if 'restaurant' in elemento:

    contador+=1

print(f"Los restaurantes abarcan el {round(contador*100/len(lista_categorias_sin_duplicados),2)}% de las categorías")


Categorías sin duplicados: ['Art gallery', 'Dry wall contractor', 'Sports massage therapist', 'Tile store', 'Basketball club', 'Leather coats store', 'Public relations firm', 'Central American restaurant', 'Taxicab Stand', 'Wedding dress rental service', 'Data entry service', 'Debt collecting', 'Law book store', 'Gardening products and services', 'Deck builder', 'Professional organizer', 'Textile mill', 'Racquetball club', 'RV dealer', 'Water works equipment supplier', 'Mental health', 'Curtain supplier and maker', 'Timeshare agency', 'Dog cafe', 'Pony ride service', 'Fruits wholesaler', 'Gasket manufacturer', 'Archaeological site', 'Mobile home park', 'Emergency room', 'Landscape lighting designer', 'Plastic surgeon', 'Gay bar', 'Data recovery service', 'Wedding bakery', 'Market', 'Salvage dealer', 'Services', 'Mercedes-Benz dealer', 'Mens tailor', 'Orthodox church', 'Computer accessories store', 'Educational testing service', 'Haunted house', 'Haute couture fashion house', 'Olive oil

Es de interés conocer cuantos de los registros tienen la cadena 'restaurant' en alguna de sus categorías, esto es lo mismo que responder a la pregunta: ¿Cuántos de estos negocios son restaurantes?.

In [15]:
df_aux = metadata_sitios.withColumn("category", col("category").cast("string"))

# Filtra y cuenta las filas que contienen 'restaurant' en la columna 'category'
contador_restaurant = df_aux.filter(col("category").contains('restaurant')).count()


# Imprime el resultado
print("El número de restaurantes es:", contador_restaurant)

El número de restaurantes es: 152198


Esto quiere decir que aproximadamente el 5% de los registros del Dataset "metadata_sitios" corresponde a restaurantes, conocer esto es de bastante utilidad para dimensionar el tamaño de los archivos que vamos a normalizar y cargar en nuestra base de datos.

In [46]:
df_aux = metadata_sitios.withColumn("category", col("category").cast("string"))

# Filtra y cuenta las filas que contienen 'Vegetarian restaurant' en la columna 'category'
contador_vegetarian = df_aux.filter(col("category").contains('Vegetarian restaurant')).count()


# Imprime el resultado
print("El número de restaurantes vegetarianos es:", contador_vegetarian)
print(f"Representa el: {round(contador_vegetarian*100/contador_restaurant,2)}% de los restaurantes")

El número de restaurantes vegetarianos es: 2775
Representa el: 1.82% de los restaurantes


Proseguiremos a explorar los demás DataFrames, analizando los datos nulos de manera similar.




In [16]:
#Se leen los reviews de Nevada

reviews_Nevada=[]


bandera=True

i=1

while bandera:

    try:




        archivo = spark.read.json(f"/content/drive/My Drive/Google Maps/reviews-estados/review-Nevada/{i}.json")



        reviews_Nevada.append(archivo)

        i+=1


    except:

        bandera=False


df_final = reviews_Nevada[0]

for dataframe in reviews_Nevada[1:]:

  df_final = df_final.unionByName(dataframe)


reviews_Nevada=df_final



In [17]:
reviews_Nevada.show()

+--------------------+--------------------+--------------------+------+----+--------------------+-------------+--------------------+
|             gmap_id|                name|                pics|rating|resp|                text|         time|             user_id|
+--------------------+--------------------+--------------------+------+----+--------------------+-------------+--------------------+
|0x80c8c5cdced316b...|              CDL CO|[{[https://lh5.go...|     5|NULL|No frills smaller...|1622740504476|10284317018302631...|
|0x80c8c5cdced316b...|           So_ Blind|                NULL|     1|NULL|Save yourself the...|1573159413820|11495439213790774...|
|0x80c8c5cdced316b...|         Greg Hansen|                NULL|     3|NULL|I passed two Star...|1515251163362|11652343333008911...|
|0x80c8c5cdced316b...|        Brett Powell|                NULL|     3|NULL|It does the job a...|1506206762095|10330405379128112...|
|0x80c8c5cdced316b...|              M Aris|                NULL|     

In [18]:
#Muestra el número de filas del DataFrame
reviews_Nevada.count()

1800000

In [19]:
conteo_nulos(reviews_Nevada)

+-------+----+-------+------+-------+------+----+-------+
|gmap_id|name|   pics|rating|   resp|  text|time|user_id|
+-------+----+-------+------+-------+------+----+-------+
|      0|   0|1732682|     0|1575071|735147|   0|      0|
+-------+----+-------+------+-------+------+----+-------+



En este caso las columnas 'pics' y 'resp' tienen muchos datos nulos como para ser consideradas, de manera similar ocurre con los otros dos Estados. También nos damos cuenta que la columna 'gmap_id' jugará un papel clave para identificar cada negocio en el Dataset de metadata_sitios

In [20]:
#Se leen los reviews de Florida

reviews_Florida=[]


bandera=True

i=1

while bandera:

    try:




        archivo = spark.read.json(f"/content/drive/My Drive/Google Maps/reviews-estados/review-Florida/{i}.json")



        reviews_Florida.append(archivo)

        i+=1


    except:

        bandera=False


df_final = reviews_Florida[0]

for dataframe in reviews_Florida[1:]:

  df_final = df_final.unionByName(dataframe)


reviews_Florida=df_final

In [21]:
reviews_Florida.show()

+--------------------+--------------------+----+------+--------------------+--------------------+-------------+--------------------+
|             gmap_id|                name|pics|rating|                resp|                text|         time|             user_id|
+--------------------+--------------------+----+------+--------------------+--------------------+-------------+--------------------+
|0x8893863ea87bd5d...|     Julie A. Gerber|NULL|     1|{Thank you for th...|Update: Their “re...|1628003250740|10147185615514872...|
|0x8893863ea87bd5d...|    Martin Sheffield|NULL|     5|{Thank you for re...|He's a knowledgea...|1595031217005|11547723478903832...|
|0x8893863ea87bd5d...|        Brian Truett|NULL|     5|                NULL|Best doctor I've ...|1522924253567|10180501024489283...|
|0x8893863ea87bd5d...|            Tina Sun|NULL|     1|                NULL|I was told he is ...|1467907819586|10634442288149374...|
|0x8893863ea87bd5d...|        James Haynes|NULL|     5|              

In [22]:
reviews_Florida.count()

2850000

In [23]:
conteo_nulos(reviews_Florida)

+-------+----+-------+------+-------+-------+----+-------+
|gmap_id|name|   pics|rating|   resp|   text|time|user_id|
+-------+----+-------+------+-------+-------+----+-------+
|      0|   0|2745810|     0|2394601|1079510|   0|      0|
+-------+----+-------+------+-------+-------+----+-------+



In [24]:
#Se leen los reviews de Virginia

reviews_Virginia=[]


bandera=True

i=1

while bandera:

    try:




        archivo = spark.read.json(f"/content/drive/My Drive/Google Maps/reviews-estados/review-Virginia/{i}.json")



        reviews_Virginia.append(archivo)

        i+=1


    except:

        bandera=False


df_final = reviews_Virginia[0]

for dataframe in reviews_Virginia[1:]:

  df_final = df_final.unionByName(dataframe)


reviews_Virginia=df_final

In [25]:
reviews_Virginia.show()

+--------------------+----------------+----+------+--------------------+--------------------+-------------+--------------------+
|             gmap_id|            name|pics|rating|                resp|                text|         time|             user_id|
+--------------------+----------------+----+------+--------------------+--------------------+-------------+--------------------+
|0x89b1af6550d3359...|      Jessie Lee|NULL|     5|                NULL|I loved Rita!! Sh...|1581455645898|11160360506098004...|
|0x89b1af6550d3359...|  Brittany Janis|NULL|     2|{Thank you for yo...|Expensive to do t...|1612044004790|10627868194435459...|
|0x89b1af6550d3359...|    Brandy Monge|NULL|     5|                NULL|She was very help...|1580144842073|10756289959510173...|
|0x89b1af6550d3359...|     Lisa Adkins|NULL|     4|                NULL|       Tracey Spadey|1612916442287|10007413172911083...|
|0x89b1af6550d3359...|     Bill Bercaw|NULL|     2|                NULL|I waited 1.5 hour...|1549

In [26]:
reviews_Virginia.count()

1662059

In [27]:
conteo_nulos(reviews_Virginia)

+-------+----+-------+------+-------+------+----+-------+
|gmap_id|name|   pics|rating|   resp|  text|time|user_id|
+-------+----+-------+------+-------+------+----+-------+
|      0|   0|1616020|     0|1437240|712868|   0|      0|
+-------+----+-------+------+-------+------+----+-------+



Hasta acá se han estudiado los Datasets que pertenecen a las reseñas de google, ahora se estudiarán los Datasets referentes a Yelp.

In [28]:
#Se lee el archivo business de yelp

with open("/content/drive/My Drive/Yelp/business.pkl", 'rb') as archivo:

        business_yelp = pickle.load(archivo)


In [29]:
business_yelp.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
business_yelp.shape[0]

150346

In [31]:
def contar_nulos(dataframe):

    # Obtener la cantidad de valores nulos por columna
  nulos_por_columna = dataframe.isnull().sum()

  print("Cantidad de valores nulos por columna:\n", nulos_por_columna)

In [ ]:
contar_nulos(business_yelp)

Cantidad de valores nulos por columna:
 business_id          0
name                 0
address              0
city                 0
state                3
postal_code          0
latitude             0
longitude            0
stars                0
review_count         0
is_open              0
attributes       13744
categories         103
hours            23223
business_id     150341
name            150341
address         150341
city            150341
state           150341
postal_code     150341
latitude        150341
longitude       150341
stars           150341
review_count    150341
is_open         150341
attributes      150341
categories      150341
hours           150341
dtype: int64


In [32]:
#Elimina las columnas repetidas que tienen casi todos sus datos nulos
business_yelp=business_yelp.iloc[:,0:14]

Este Dataset 'business yelp' tiene muchas columnas repetidas, en su mayoría los datos de esas copias son nulos, para nuestro trabajo deben ser eliminadas. Es importante conocer el número de negocios que pertenecen a los Estados que son objeto de nuestro estudio.

In [33]:
business_yelp[(business_yelp['state']=='NV') | ((business_yelp['state']=='FL') | (business_yelp['state']=='VA'))].shape[0]

34044

In [34]:
proporcion= round(34044*100/150346,2)

print(f"El porcentaje de datos que corresponde a alguno de los tres Estados es: {proporcion}%")

El porcentaje de datos que corresponde a alguno de los tres Estados es: 22.64%


In [38]:
contador= 0
for elemento in business_yelp['categories']:

  try:

    if 'Restaurants' in elemento:

      contador+=1

  except:

    continue

print("La cantidad total de restaurantes es: ", contador)
print(f"Representan un {round(contador*100/150346,2)}% de los negocios")

La cantidad total de restaurantes es:  52268
Representan un 34.77% de los negocios


In [ ]:
#Se lee el archivo checkin de yelp

checkin_yelp=[]

with open("/content/drive/My Drive/Yelp/checkin.json","r", encoding= 'utf-8') as filejson:


                # Procesa cada línea del archivo como un objeto JSON
                for linea in filejson:

                        # Intenta cargar la línea como un objeto JSON
                        objeto = json.loads(linea)

                        checkin_yelp.append(objeto)

checkin_yelp=pd.DataFrame(checkin_yelp)


In [ ]:
checkin_yelp.head(5)

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


In [ ]:
checkin_yelp.shape[0]

131930

In [ ]:
contar_nulos(checkin_yelp)

Cantidad de valores nulos por columna:
 business_id    0
date           0
dtype: int64


In [ ]:
#Se lee el archivo review de yelp

review_yelp = spark.read.json("/content/drive/My Drive/Yelp/review.json")

In [ ]:
review_yelp.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

In [ ]:
review_yelp.count()

6990280

In [ ]:
conteo_nulos(review_yelp)

+-----------+----+----+-----+---------+-----+----+------+-------+
|business_id|cool|date|funny|review_id|stars|text|useful|user_id|
+-----------+----+----+-----+---------+-----+----+------+-------+
|          0|   0|   0|    0|        0|    0|   0|     0|      0|
+-----------+----+----+-----+---------+-----+----+------+-------+



En este caso no hay valores nulos, lo que es bastante positivo para nuestro sistema de recomendación y para facilitar la limpieza de datos.

In [ ]:
#Se lee el archivo tip de yelp

tip_yelp=[]

with open("/content/drive/My Drive/Yelp/tip.json","r", encoding= 'utf-8') as filejson:


                # Procesa cada línea del archivo como un objeto JSON
                for linea in filejson:

                        # Intenta cargar la línea como un objeto JSON
                        objeto = json.loads(linea)

                        tip_yelp.append(objeto)

tip_yelp=pd.DataFrame(tip_yelp)


In [ ]:
tip_yelp.head(5)

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [ ]:
tip_yelp.shape[0]

908915

In [ ]:
contar_nulos(tip_yelp)

Cantidad de valores nulos por columna:
 user_id             0
business_id         0
text                0
date                0
compliment_count    0
dtype: int64


In [ ]:
#Se lee el archivo user de yelp

user_yelp= pd.read_parquet("/content/drive/My Drive/Yelp/user.parquet")

In [ ]:
user_yelp.head(5)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


In [ ]:
user_yelp.shape[0]

2105597

In [ ]:
contar_nulos(user_yelp)

Cantidad de valores nulos por columna:
 user_id               0
name                  0
review_count          0
yelping_since         0
useful                0
funny                 0
cool                  0
elite                 0
friends               0
fans                  0
average_stars         0
compliment_hot        0
compliment_more       0
compliment_profile    0
compliment_cute       0
compliment_list       0
compliment_note       0
compliment_plain      0
compliment_cool       0
compliment_funny      0
compliment_writer     0
compliment_photos     0
dtype: int64


Como se pudo observar, los datos de Yelp no necesitan mayor tratamiento en cuanto a datos faltantes.